# Getting started with LangTrace and Langchain 

## 1. Setup LangTrace 

* Sign-up for a free account on [LangTrace](https://langtrace.ai/.com "Title").

* Create a Project and generate an API Key

* <span style="color:red"> Note that for this tutorial you will also sign up and get an API key from [OPENAI](https://OPENAI.COM/ "OPEN AI") if you havent done so already. </span>


## 2. Install packages and setup environment

In [ ]:
pip install -U langtrace-python-sdk langchain langchain-chroma langchainhub

In [ ]:

import os
os.environ['LANGTRACE_API_KEY'] = "YOUR_LANGTRACE_API_KEY"
os.environ['OPENAI_API_KEY'] = "YOUR_OPEN_AI_KEY"

<br>

## 3. Imports

In [ ]:
from langtrace_python_sdk import langtrace # Must precede any llm module imports
langtrace.init(api_key = os.environ['LANGTRACE_API_KEY'])
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

## 4. Create a vector store index and query it

### We will be loading a document from the assets folder to create embeddings and query it

In [ ]:
loader = TextLoader("assets/soccer_rules.txt")
docs = loader.load()

In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

### Retrieve and generate using the relevant snippets of the document.

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

### Format our document ,setup a rag chain and query

In [ ]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Offside??")

## You can now view your traces on the Langtrace dashboard

![image info](./assets/image1.png)